<a href="https://colab.research.google.com/github/deynabaevax/groupm-graduation/blob/developing/code/eda/eda_jumbo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# EDA 
### Jumbo Data

### Import the libraries

In [ ]:
!pip install emoji

In [3]:
import pandas as pd
import numpy as np
import re
import string
import emoji

import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation

import warnings
warnings.filterwarnings("ignore")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


### Import the data set

In [28]:
df = pd.read_excel("/content/drive/MyDrive/datasets/jumbo-data.xlsx")

In [ ]:
df.head()

In [34]:
# rename the columns for easy typing
df.rename(columns={"Case Number": "id", "Case Description": "descr", "Case Origin": "source"}, inplace=True)

In [35]:
df.shape

(95921, 3)

In [ ]:
df.info()

In [ ]:
df.describe().T

In [ ]:
df.isnull().sum()

In [ ]:
df["descr"].isnull().values.any()

In [ ]:
# check for duplicates in the 'query' column
duplicates = df.duplicated(subset=["id" ,"descr"])
print("Number of duplicates:", duplicates.sum())

In [ ]:
# create a boolean mask of null values
mask = df.isnull().any(axis=1)

# select the rows that have at least one null value
rows_with_null = df[mask]

# print the selected rows
print(rows_with_null)

In [ ]:
df["descr"][92513]

In [ ]:
df["descr"][92517]

In [ ]:
df["descr"][60314]

In [30]:
# the above row seems usless, so I'll remove it
id = [60314]
df = df.drop(id, axis=0)

In [ ]:
df["descr"][92519] 

In [ ]:
df["descr"][92515]

In [ ]:
 df.isnull().sum()[df.isnull().sum()>0]

In [53]:
data = df.dropna()